In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
height = 120
width = 120
batch_size = 64

In [3]:
ds_train = tf.keras.utils.image_dataset_from_directory(
  'data/',
  validation_split=0.2,
  subset="training",
  shuffle=True,
  #seed=123,
  image_size=(height, width),
  batch_size=batch_size
)
ds_val = tf.keras.utils.image_dataset_from_directory(
  'data/',
  validation_split=0.2,
  subset="validation",
  shuffle=True,
  seed=123,
  image_size=(height, width),
  batch_size=batch_size
)

Found 2269 files belonging to 2 classes.


In [4]:
class_names = ds_train.class_names
print(class_names)

['Bicycle', 'Other']


In [5]:
def normalize_img(image,label):
    return tf.cast(image,tf.float32)/255.0, label

In [6]:
plt.figure(figsize=(10, 10))
for images, labels in ds_train.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")
plt.figure(figsize=(10, 10))
for images, labels in ds_val.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = ds_train.map(normalize_img, num_parallel_calls = AUTOTUNE)
ds_val = ds_val.map(normalize_img, num_parallel_calls = AUTOTUNE)

In [8]:
def Model():
    inputs = keras.Input(shape=(120,120,3))
    
    x = layers.Conv2D(64,3,padding='valid')(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(128,3,padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(256,3,padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(256,3,padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(256,3,padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    
    x = layers.Flatten()(x)
    #kernel_regularizer=regularizers.l2(.01)
    x = layers.Dense(1024)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    
    x = layers.Dense(256)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    
    #x = layers.Dropout(.5)(x)
    outputs = layers.Dense(2)(x)
    
    model = keras.Model(inputs=inputs,outputs=outputs)
    return model

model = Model()

print(model.summary())

In [2]:
#created model99.8 by going through entire training set, then used transfer learning to create
#models for sub-categories - explained in demo - 

model = keras.models.load_model('models/model99.8')
base_inputs = model.layers[0].input
base_outputs = model.layers[-2].output
x = layers.Dense(256)(base_outputs)
x = layers.BatchNormalization(name = "bn1")(x)
x = keras.activations.relu(x)
x = layers.Dense(96)(x)
x = layers.BatchNormalization(name = "bn2")(x)
x = keras.activations.relu(x)
final_outputs = layers.Dense(2)(x)
new_model = keras.Model(inputs=base_inputs,outputs=final_outputs)
new_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)
new_model.fit(ds_train, batch_size=64, epochs=3)
new_model.evaluate(ds_train,batch_size=64)

In [14]:
model = keras.models.load_model('models/model99.8')
model.save('models/model99.8')

INFO:tensorflow:Assets written to: models/model99.8\assets


INFO:tensorflow:Assets written to: models/model99.8\assets
